In [1]:
%%capture
import waymax
import numpy as np
import mediapy
from tqdm import tqdm
import dataclasses
import pickle
import tensorflow as tf

from waymax import config as _config
from waymax import dataloader
from waymax import datatypes
from waymax import visualization

2024-11-08 12:33:24.115750: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 12:33:24.123011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 12:33:24.130411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 12:33:24.132654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 12:33:24.139140: I tensorflow/core/platform/cpu_feature_guar

### Configurations

In [2]:
data_path = '/home/emerge/daphne/gpudrive/data/processed/validation'

### Get Waymo validation files

Steps

```bash
gcloud auth login
```

```
gsutil -m cp -r gs://waymo_open_dataset_motion_v_1_1_0/uncompressed/scenario/validation/ data/raw
```


In [3]:
from pathlib import Path

filenames = [
    p.name for p in Path(data_path).iterdir()
]

scenario_ids = [p.split("_")[-1].split(".")[0] for p in filenames]

In [4]:
#'11671609ebfa3185' in scenario_ids

### Search through Waymax files, find a match and store the scenario

- Using the WOMD validation set from version 1.1.0. See [link](https://github.com/waymo-research/waymax/blob/720f9214a9bf79b3da7926497f0cd0468ca3e630/waymax/config.py#L269)

In [8]:
def _preprocess(serialized: bytes) -> dict[str, tf.Tensor]:
    womd_features = dataloader.womd_utils.get_features_description(
        include_sdc_paths=data_config.include_sdc_paths,
        max_num_rg_points=data_config.max_num_rg_points,
        num_paths=data_config.num_paths,
        num_points_per_path=data_config.num_points_per_path,
    )
    womd_features['scenario/id'] = tf.io.FixedLenFeature([1], tf.string)

    deserialized = tf.io.parse_example(serialized, womd_features)
    parsed_id = deserialized.pop('scenario/id')
    deserialized['scenario/id'] = tf.io.decode_raw(parsed_id, tf.uint8)

    return dataloader.preprocess_womd_example(
        deserialized,
        aggregate_timesteps=data_config.aggregate_timesteps,
        max_num_objects=data_config.max_num_objects,
    )

def _postprocess(example: dict[str, tf.Tensor]):
    scenario = dataloader.simulator_state_from_womd_dict(example)
    scenario_id = example['scenario/id']
    return scenario_id, scenario

In [9]:
data_config = dataclasses.replace(
    _config.WOD_1_1_0_VALIDATION, 
    max_num_objects=32,
)
#v1.2.0
data_iter = dataloader.simulator_state_generator(config=data_config)

# Caution: This takes about a minute
scenario = next(data_iter)

In [10]:
# Get a single scenario
scene_id_iter = dataloader.get_data_generator(
    data_config, _preprocess, _postprocess
)

In [11]:
scenario_id, scenario = next(scene_id_iter)
scenario_id = scenario_id.tobytes().decode('utf-8')
print(scenario_id)

40b9ad6ddb5a066b


In [12]:
counter = 0

# Get the unique ID string for a scenario
for scenario_id, scenario in scene_id_iter:
    # Decode bytes and return the scenario ID
    scenario_id = scenario_id.tobytes().decode('utf-8')

    if scenario_id == '2bd577a009790706':
    # if scenario_id in scenario_ids:
        print(f"Found scenario: {scenario_id}! Saving to file...")
        scenario_path = f'waymax_scenario_{scenario_id}.pkl'
        with open(scenario_path, 'wb') as f:    
            pickle.dump(scenario, f)
        counter += 1
        if counter == 1:
            break
    
else: 
    print('Done')

Found scenario: 2bd577a009790706! Saving to file...


In [14]:
import os 
import json

TARGET_ID = '2bd577a009790706'

for file_name in os.listdir(data_path):
    # Check if file name contains the target id and ends with .json
    if TARGET_ID in file_name:
        file_path = os.path.join(data_path, file_name)
        print('found!')
        with open(file_path, 'r') as file:
            data = json.load(file)
        # Store the data in the specified output path
        with open(file_name, 'w') as output_file:
            json.dump(data, output_file, indent=4)

found!


In [8]:
os.listdir(data_path)

['tfrecord-00003-of-00150_6cdfa1c30913f8e2.json',
 'tfrecord-00018-of-00150_bd71015c32840e34.json',
 'tfrecord-00013-of-00150_ba0be9573301433b.json',
 'tfrecord-00005-of-00150_9ddede7398a0e27b.json',
 'tfrecord-00013-of-00150_dd0504e6cc66df91.json',
 'tfrecord-00009-of-00150_7aabb60366a97c68.json',
 'tfrecord-00016-of-00150_3b6bb08396ff1786.json',
 'tfrecord-00010-of-00150_f9aec5b89968328f.json',
 'tfrecord-00005-of-00150_6f4d231ddc9261e0.json',
 'tfrecord-00011-of-00150_92f84fb0e0c30630.json',
 'tfrecord-00018-of-00150_bbfb31d23dbf0ce.json',
 'tfrecord-00008-of-00150_23c5af73cb7986de.json',
 'tfrecord-00006-of-00150_da0fe29904e7a987.json',
 'tfrecord-00019-of-00150_c86dd2eea30f6b2d.json',
 'tfrecord-00051-of-00150_b62ca36ac691ca9.json',
 'tfrecord-00011-of-00150_e8458513b6f38a55.json',
 'tfrecord-00014-of-00150_4c5cae1d00d12e34.json',
 'tfrecord-00009-of-00150_15d729719783f849.json',
 'tfrecord-00013-of-00150_c0548991871902a6.json',
 'tfrecord-00007-of-00150_73ff5ea2ab50a4fe.json',
 '

In [9]:
scenario_id

'ab2a72c63f8fd589'

In [10]:
'ab2a72c63f8fd589' in scenario_ids

True